<a href="https://colab.research.google.com/github/CISC452MLProject/CISC-452-Group-22/blob/main/group22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom Trained Model

In [ ]:
import numpy
import torch
import mmpose

print(torch.__version__)
print(numpy.__version__)
print(mmpose.__version__)

2.1.3


### Convert the provided files (skeletons, metadata.txt) into a format compatible with MMPose. MMPose expects COCO-style JSON annotations for training.

In [7]:
import json
import os

def parse_metadata(metadata_path):
    metadata = {}
    with open(metadata_path, 'r') as f:
        for line in f:
            parts = line.strip().split(';')
            metadata[parts[0]] = {
                'id_climber': parts[1],
                'date': parts[2],
                'run_number': parts[3],
                'url': parts[4],
                'start': float(parts[5]),
                'end': float(parts[6]),
                'time_sec': float(parts[7]),
                'time_frames': int(parts[8]),
                'finished': int(parts[9]),
                'side': parts[10],
                'fps': float(parts[11])
            }
    return metadata

def parse_skeletons(skeletons_dir):
    skeleton_data = {}
    for file_name in os.listdir(skeletons_dir):
        run_id = file_name.replace('.txt', '')
        skeleton_data[run_id] = []
        with open(os.path.join(skeletons_dir, file_name), 'r') as f:
            for line in f:
                parts = line.strip().split('#')
                frame_number = int(parts[0])
                keypoints = parts[1].split(';')
                keypoints = [float(coord) for kp in keypoints for coord in kp.split(',')]
                skeleton_data[run_id].append({
                    'frame_number': frame_number,
                    'keypoints': keypoints
                })
    return skeleton_data

def generate_coco_json(images_dir, skeletons, metadata, output_json):
    coco_data = {
        'images': [],
        'annotations': [],
        'categories': [{
            'id': 1,
            'name': 'person',
            'keypoints': [f'j{i+1}' for i in range(16)],
            'skeleton': [[i, i+1] for i in range(1, 16)]  # Assuming a simple linear skeleton structure
        }]
    }

    annotation_id = 1
    for run_id, frames in skeletons.items():
        climber_metadata = metadata.get(run_id, {})
        for frame in frames:
            image_id = len(coco_data['images']) + 1
            frame_number = frame['frame_number']
            image_path = os.path.join(images_dir, f'frame{frame_number}.jpg')
            coco_data['images'].append({
                'id': image_id,
                'file_name': image_path,
                'height': 720,  # Adjust based on your actual image size
                'width': 1280
            })
            coco_data['annotations'].append({
                'id': annotation_id,
                'image_id': image_id,
                'category_id': 1,
                'keypoints': frame['keypoints'],
                'num_keypoints': sum(1 for x in frame['keypoints'][2::3] if x > 0)
            })
            annotation_id += 1

    with open(output_json, 'w') as f:
        json.dump(coco_data, f, indent=4)

# Paths
images_dir = 'img'
skeletons_dir = 'data_unprocessed/skeletons'
metadata_file = 'data_unprocessed/metadata.txt'
output_json = 'climbData.json'
test_json = 'test.json'

# Process
metadata = parse_metadata(metadata_file)
skeletons = parse_skeletons(skeletons_dir)

with open(test_json, 'w') as f:
    json.dump(skeletons, f, indent=4)


# generate_coco_json(images_dir, skeletons, metadata, output_json)


ValueError: could not convert string to float: 'NULL'